In [2]:
startPage = "twitter.com"
import http.client as hc
import re
from bs4 import BeautifulSoup
import queue
import json

jsonDecoder = json.JSONDecoder()

def robot_valid_href(href):
    if not href:
        return False
    # Note still must abide by crawl-delay = 1
    valid = True
    valid &= (not not re.compile("twitter.com").search(href)) or href[0] == "/"
    valid &= not re.compile("/search/realtime").search(href)
    valid &= not re.compile("/search/users").search(href)
    valid &= not re.compile(r"/search/.+/grid").search(href)

    valid &= not re.compile(r"/.+/followers").search(href)
    valid &= not re.compile(r"/.+/following").search(href)

    valid &= not re.compile("/oauth").search(href)
    valid &= not re.compile("/1/oauth").search(href)

    valid &= not re.compile("/i/streams").search(href)
    valid &= not re.compile("/i/hello").search(href)
    
    valid &= not re.compile("/account/deactivated").search(href)
    valid &= not re.compile("/settings/deactivated").search(href)
    return href and valid

In [3]:
def parsePage(url, basePage=startPage):
    conn1 = hc.HTTPSConnection(basePage)
    conn1.set_debuglevel(0)
    conn1.request("GET", url)
    resp = conn1.getresponse()
    soup = BeautifulSoup(resp.read(), "html.parser")
    return soup

In [4]:
def scrapeUrls(soup):
    urls = []
    for tag in soup.find_all(href=robot_valid_href):
        urls.append(tag["href"])
    for i in range(len(urls)):
        if urls[i][0] == "/":
            urls[i] = "https://" + startPage + urls[i]
    return urls

In [5]:
def tweetFilter(tag):
    return tag.has_attr("data-item-type") and tag["data-item-type"] == "tweet"

def scrapeTweet(soup):
    tweets = []
    for tag in soup.find_all(tweetFilter):
        # Get immediate subclass with tweet data
        dataTag = tag.find("div")
        if (not dataTag.has_attr("data-reply-to-users-json")):
            # print("\n\n\nERROR\n\n\n", tag, "\n\n\n", dataTag)
            continue
        tweetTag = dataTag.find("p", class_=re.compile("tweet-text"))
        # print(dataTag["data-reply-to-users-json"])
        
        userJsonData = jsonDecoder.decode(dataTag["data-reply-to-users-json"])
        # print(userJsonData)
        # print(dataTag["data-reply-to-users-json"])
        
        
        permaPath = dataTag["data-permalink-path"]
        # print(permaPath)
        tweetText = tweetTag.get_text()
        primaryUser = userJsonData[0]
        tweets.append((primaryUser["id_str"], primaryUser["screen_name"], primaryUser["name"], permaPath, tweetText))
        # print(tweetTag.get_text(), "\n\nNEW_TWEET\n")
    return tweets
        
    

In [6]:
def processUrl(url):
    soup = parsePage(url)
    urls = scrapeUrls(soup)
    tweetLists = scrapeTweet(soup)

In [30]:
def crawlAround():
    # Load the urlFrontier into a queue
    urlFrontier = open("crawlData/urlFrontier.txt","r")
    urlFrontierQueue = queue.SimpleQueue()
    for line in urlFrontier:
        urlFrontierQueue.put(line)
    urlFrontier.close()
    # Load the urlRepo into a set
    urlRepo = open("crawlData/urlRepo.txt", "r")
    urlRepoSet = set({})
    for line in urlRepo:
        line = line.replace("\n","") # Have to trim the end since set is so specific
        urlRepoSet.add(line)
    urlRepo.close()
    # Start crawling process
    print(urlFrontierQueue.qsize())
    print(urlRepoSet)
    

In [31]:
crawlAround()

4
{'https://twitter.com/search?q=Hello'}
